# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

# 4 - ALS Model - Reduced Data

This time, as explored in the EDA NB, let's consider removing customers who we feel have too few or too many purchases to influence the model in the intended way.

Examples:
- Too few - Customers who have only bought 1 comic (series).
- Too many - Customers with > 1000 series (for example, think all eBay customers are rolled into one account number).

# Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2  # 1 would be where you need to specify the files
#%aimport data_fcns

import pandas as pd  # dataframes
import os
import time
import numpy as np

# Data storage
from sqlalchemy import create_engine  # SQL helper
import psycopg2 as psql  #PostgreSQL DBs

# import necessary libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import (StructType, StructField, IntegerType
#                                ,FloatType, LongType, StringType)
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, lit, isnan, when, count
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
import sys

In [3]:
sys.path.append('..')

In [4]:
# Custom
import data_fcns as dfc
import keys  # Custom keys lib
import comic_recs as cr

In [5]:
# instantiate SparkSession object
spark = pyspark.sql.SparkSession.builder.master("local[*]").getOrCreate()
# spark = SparkSession.builder.master("local").getOrCreate()

## Import the data

There is way to directly hit PostgreSQL through JDBC, but I don't know how to do that yet. So have worked around by saving the candidate dataset to JSON, and then will use that as input to Spark.


In [6]:
# We have previously created a version of the transactions table and filtered it down.
trans = spark.read.json('raw_data/trans_filtered_floor.json')

In [7]:
# Persist the data
trans.persist()

DataFrame[account_num: string, comic_title: string, date_sold: bigint, item_id: string, publisher: string, qty_sold: bigint, title_and_num: string]

In [8]:
print(trans.count(), len(trans.columns))

288062 7


In [9]:
# check schema
trans.printSchema()

root
 |-- account_num: string (nullable = true)
 |-- comic_title: string (nullable = true)
 |-- date_sold: long (nullable = true)
 |-- item_id: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- qty_sold: long (nullable = true)
 |-- title_and_num: string (nullable = true)



### More exploration/testing

We won't be using pandas dataframes in the matrix factorization through Spark, but let's cast to one anyway as it will be easier to work with for EDA.

In [10]:
# cast to Pandas dataframe to turn timestamp data to datetime and check nulls.
trans_df = trans.select('*').toPandas()
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288062 entries, 0 to 288061
Data columns (total 7 columns):
account_num      288062 non-null object
comic_title      288062 non-null object
date_sold        288062 non-null int64
item_id          288062 non-null object
publisher        288062 non-null object
qty_sold         288062 non-null int64
title_and_num    288062 non-null object
dtypes: int64(2), object(5)
memory usage: 15.4+ MB


In [11]:
# Let's double check the data is how we expect it
trans_df.head()

account_num                     comic_title      date_sold    item_id  \
0       01033  Afterlife With Archie (Archie)  1390505789000  DCD630105   
1       01333  Afterlife With Archie (Archie)  1398947834000  DCD630105   
2       00946  Afterlife With Archie (Archie)  1399904156000  DCD630105   
3       01278  Afterlife With Archie (Archie)  1407954335000  DCD630105   
4       01212  Afterlife With Archie (Archie)  1383751270000  DCD630105   

       publisher  qty_sold                   title_and_num  
0  Archie Comics         1  Afterlife With Archie #1 2nd P  
1  Archie Comics         1  Afterlife With Archie #1 2nd P  
2  Archie Comics         1  Afterlife With Archie #1 2nd P  
3  Archie Comics         1  Afterlife With Archie #1 2nd P  
4  Archie Comics         1  Afterlife With Archie #1 2nd P

In [12]:
trans_df['dt'] = pd.to_datetime(trans_df['date_sold'], unit='ms')

Yes. Reverse-confirmed versus the original transactions dataframe in the other notebook that this datetime is correct. 

### Data Prep for ALS

Let's aggregate the data to the two columns we need:
- `account_num` - This is the identifier for individual customers.


- `comic_title` - The comic. Represents individual volumes/runs of a comic.


- `score` - We need to figure out what we want to use to act as a `score`. If these were Amazon items then review scores would be natural fit; but we don't have that. We can maybe use a binary flag of `bought`/`not bought`. Or we can use the `qty_sold`. This might be interesting in that it might capture some interesting behavior from comic 'collectors/speculators'. Since this is first pass, I'm curious as to what `qty_sold` might do!


We only care about `account_num`, `comic_title` and `qty_sold`.

In [13]:
comics_sold = trans[['account_num', 'comic_title', 'qty_sold']]
comics_sold.persist()

DataFrame[account_num: string, comic_title: string, qty_sold: bigint]

In [14]:
comics_sold = comics_sold.withColumn('bought', lit(1))

In [15]:
comics_sold.show(10)

+-----------+--------------------+--------+------+
|account_num|         comic_title|qty_sold|bought|
+-----------+--------------------+--------+------+
|      01033|Afterlife With Ar...|       1|     1|
|      01333|Afterlife With Ar...|       1|     1|
|      00946|Afterlife With Ar...|       1|     1|
|      01278|Afterlife With Ar...|       1|     1|
|      01212|Afterlife With Ar...|       1|     1|
|      00877|Afterlife With Ar...|       1|     1|
|      01155|Afterlife With Ar...|       1|     1|
|      01099|Afterlife With Ar...|       1|     1|
|      00127|Afterlife With Ar...|       1|     1|
|      01033|Afterlife With Ar...|       1|     1|
+-----------+--------------------+--------+------+
only showing top 10 rows



In [16]:
comics_sold = trans[['account_num', 'comic_title', 'qty_sold']]
comics_sold.persist()

DataFrame[account_num: string, comic_title: string, qty_sold: bigint]

In [17]:
total_comics_sold = comics_sold.groupBy(['account_num', 'comic_title'
                                         ]).agg({'qty_sold': 'sum'})
total_comics_sold.persist()

DataFrame[account_num: string, comic_title: string, sum(qty_sold): bigint]

Ok, let's take a look at the results.

In [18]:
total_comics_sold.show(10)

+-----------+--------------------+-------------+
|account_num|         comic_title|sum(qty_sold)|
+-----------+--------------------+-------------+
|      00487|Captain Swing (Av...|            2|
|      00029|God Is Dead (Avatar)|            7|
|      01260| Providence (Avatar)|            1|
|      00172|   Supergod (Avatar)|            3|
|      00032|Big Trouble In Li...|           11|
|      00130|Six Gun Gorilla (...|            3|
|      00742|      Unsound (Boom)|            6|
|      01406|Age of Reptiles A...|            5|
|      00291|Angel & Faith (Da...|            2|
|      00032|Baltimore Curse B...|            5|
+-----------+--------------------+-------------+
only showing top 10 rows



In [19]:
print(total_comics_sold.count(), len(total_comics_sold.columns))

40174 3


In [20]:
total_comics_sold = total_comics_sold.withColumn('bought', lit(1))

I don't like that default column name. Let's fix that to be `qty_sold` again.

In [21]:
total_comics_sold.show(10)

+-----------+--------------------+-------------+------+
|account_num|         comic_title|sum(qty_sold)|bought|
+-----------+--------------------+-------------+------+
|      00487|Captain Swing (Av...|            2|     1|
|      00029|God Is Dead (Avatar)|            7|     1|
|      01260| Providence (Avatar)|            1|     1|
|      00172|   Supergod (Avatar)|            3|     1|
|      00032|Big Trouble In Li...|           11|     1|
|      00130|Six Gun Gorilla (...|            3|     1|
|      00742|      Unsound (Boom)|            6|     1|
|      01406|Age of Reptiles A...|            5|     1|
|      00291|Angel & Faith (Da...|            2|     1|
|      00032|Baltimore Curse B...|            5|     1|
+-----------+--------------------+-------------+------+
only showing top 10 rows



In [22]:
total_comics_sold = total_comics_sold[['account_num', 'comic_title', 'bought']]

In [23]:
print(total_comics_sold.count(), len(total_comics_sold.columns))

40174 3


### Formatting

Sooooooo, I forgot that the values need to be numeric. So need to fix that.

#### Convert `account_id` to integer

In [24]:
to_int_udf = F.udf(dfc.make_int, IntegerType())

In [25]:
account_num_col = total_comics_sold['account_num']

In [26]:
total_comics_sold = total_comics_sold.withColumn('account_id',
                                                 to_int_udf(account_num_col))
total_comics_sold.persist()

DataFrame[account_num: string, comic_title: string, bought: int, account_id: int]

In [27]:
total_comics_sold.show(10)

+-----------+--------------------+------+----------+
|account_num|         comic_title|bought|account_id|
+-----------+--------------------+------+----------+
|      00487|Captain Swing (Av...|     1|       487|
|      00029|God Is Dead (Avatar)|     1|        29|
|      01260| Providence (Avatar)|     1|      1260|
|      00172|   Supergod (Avatar)|     1|       172|
|      00032|Big Trouble In Li...|     1|        32|
|      00130|Six Gun Gorilla (...|     1|       130|
|      00742|      Unsound (Boom)|     1|       742|
|      01406|Age of Reptiles A...|     1|      1406|
|      00291|Angel & Faith (Da...|     1|       291|
|      00032|Baltimore Curse B...|     1|        32|
+-----------+--------------------+------+----------+
only showing top 10 rows



In [28]:
print(total_comics_sold.count(), len(total_comics_sold.columns))

40174 4


Now I need to find a way to give ids to the `comic_title`. Kind of clunky, but I do have the version in PostgreSQL of the big table. I can just build an ID table up there as source of truth. I could do something on PySpark side, but then think would want to save it somewhere (e.g. the DB) anyway. So might as well do it from the top.

#### Get `comic_id`

In [29]:
comics = spark.read.json('raw_data/comics.json')
comics.persist()

DataFrame[comic_id: bigint, comic_title: string]

In [30]:
comics.count()

7202

In [31]:
comics.show(10)

+--------+--------------------+
|comic_id|         comic_title|
+--------+--------------------+
|       1|0Secret Wars (Mar...|
|       2|100 Bullets Broth...|
|       3|100 Penny Press L...|
|       4|100 Penny Press S...|
|       5|100 Penny Press T...|
|       6|100 Penny Press T...|
|       7|100th Anniversary...|
|       8|12 Reasons To Die...|
|       9|    13 Coins (Other)|
|      10|13th Artifact One...|
+--------+--------------------+
only showing top 10 rows



In [32]:
print(comics.count(), len(comics.columns))

7202 2


Now we need to join this back into `total_comics_sold`.

In [33]:
# Set aliases
tot = total_comics_sold.alias('tot')
com = comics.alias('com')

In [34]:
tot_sold_ids_only = tot.join(com.select('comic_id', 'comic_title'),
                             tot.comic_title == com.comic_title).select(
                                 'account_id', 'comic_id', 'bought')
tot_sold_ids_only.persist()
tot_sold_ids_only.show(10)

+----------+--------+------+
|account_id|comic_id|bought|
+----------+--------+------+
|       487|    1102|     1|
|        29|    2680|     1|
|      1260|    4870|     1|
|       172|    6023|     1|
|        32|     755|     1|
|       130|    5497|     1|
|       742|    6717|     1|
|      1406|     136|     1|
|       291|     260|     1|
|        32|     525|     1|
+----------+--------+------+
only showing top 10 rows



In [35]:
tot_sold_ids_only.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- comic_id: long (nullable = true)
 |-- bought: integer (nullable = false)



In [36]:
print(tot_sold_ids_only.count(), len(tot_sold_ids_only.columns))

40174 3


## Save this intermediate table.

To save work, if needed.

In [37]:
als_input_df = tot_sold_ids_only.toPandas()

In [38]:
als_input_df.shape

(40174, 3)

In [39]:
als_input_df.to_json('raw_data/als_input_filtered_floor.json',
                     orient='records',
                     lines=True)

In [40]:
!head raw_data/als_input_filtered.json

{"account_id":2247,"comic_id":995,"bought":1}
{"account_id":487,"comic_id":1102,"bought":1}
{"account_id":29,"comic_id":2680,"bought":1}
{"account_id":1260,"comic_id":4870,"bought":1}
{"account_id":172,"comic_id":6023,"bought":1}
{"account_id":2493,"comic_id":66,"bought":1}
{"account_id":52,"comic_id":116,"bought":1}
{"account_id":32,"comic_id":755,"bought":1}
{"account_id":1149,"comic_id":971,"bought":1}
{"account_id":1489,"comic_id":3503,"bought":1}
